In [14]:
import numpy as np
import json
from pathlib import Path

In [101]:
from ezc3d import c3d
c = c3d('00599_raw.c3d')
print(c['parameters']['POINT']['USED']['value'][0]);  # Print the number of points used
point_data = c['data']['points']
points_residuals = c['data']['meta_points']['residuals']
analog_data = c['data']['analogs']

data = c.c3d_swig.get_points()
print(data.shape)
# print(data)
print(data[:3,:,0:2].shape)

53
(4, 53, 440)
(3, 53, 2)


In [102]:
# FIXME: fix temp cut-off 53 to 49
# :49
# 4:
a = data[:3,:,0:]
a.shape

(3, 53, 440)

In [103]:
# transpose 2, 53, 3

In [121]:
b = np.transpose(a, (2, 1, 0))
b.shape

(440, 53, 3)

In [105]:
# reshape to 2, 53*3 and divide by 100, and round to 4 decimal places

In [106]:
# find mapping

In [107]:
labels = c['parameters']['POINT']['LABELS']['value']
len(labels), labels[:5]

(53, ['RPSI', 'LPSI', 'L3', 'STRN', 'T10'])

In [108]:
# TODO: identify and remove 4 unconnected points

In [109]:
bad_labels = [
    "RBAK",
    "LBAK",
    "LAOL",
    "RAOL"
]

In [112]:
bad_label_idxs = [labels.index(label) for label in bad_labels]
bad_label_idxs

[7, 8, 12, 22]

In [117]:
labels = [label for label in labels if not label in bad_labels]

In [118]:
b.shape

(440, 49, 3)

In [119]:
mask = [not x in bad_label_idxs for x in range(53)]

In [122]:
b = b[:,mask,:]
b.shape

(440, 49, 3)

In [123]:
d = (np.reshape(b, (b.shape[0], b.shape[1]*3))/100).round(4)
d.shape

(440, 147)

In [124]:
point_list = d[:, :].tolist()

In [125]:
json_out = {
    'frames': point_list,
    'interval': 20,
    'markers': labels
}

In [126]:
json_str = json.dumps(json_out) #, indent=3)

Path("/Users/wcz/Beanflows/All_Beans/swift4tf/motion-viewer/ezc3d_tests/00599_raw.json").write_text(json_str)

541840